In [1]:
TRAIN_SINGLE_DATASET_PATH = 'CompLex/train/lcp_single_train.tsv'
TEST_SINGLE_DATASET_PATH = 'CompLex/test-labels/lcp_single_test.tsv'
TRAIN_MULTI_DATASET_PATH = 'CompLex/train/lcp_multi_train.tsv'
TEST_MULTI_DATASET_PATH = 'CompLex/test-labels/lcp_multi_test.tsv'

LANGUAGE = 'english'

from gensim.models.fasttext import FastText
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
import numpy as np
import pandas as pd
import itertools
import re
import csv

In [2]:
def load_dataframes():
    train_single_df = pd.read_csv(TRAIN_SINGLE_DATASET_PATH, sep='\t', quoting=csv.QUOTE_NONE)
    test_single_df = pd.read_csv(TEST_SINGLE_DATASET_PATH, sep='\t', quoting=csv.QUOTE_NONE)
    train_multi_df = pd.read_csv(TRAIN_MULTI_DATASET_PATH, sep='\t', quoting=csv.QUOTE_NONE)
    test_multi_df = pd.read_csv(TEST_MULTI_DATASET_PATH, sep='\t', quoting=csv.QUOTE_NONE)
    return train_single_df, test_single_df, train_multi_df, test_multi_df

def preprocess_data(sentences, language):
    language_stopwords = set(stopwords.words(language))

    sentences = [re.sub(r'[^a-zA-Z\s]', ' ', sentence).lower() for sentence in sentences]
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
    tokenized_sentences = [
        [word for word in sentence if word not in language_stopwords]
        for sentence in tokenized_sentences
    ]

    return tokenized_sentences

def train_fasttext():
    wordnet_synset_examples = [synset.examples() for synset in wordnet.all_synsets()]
    wordnet_synset_examples = list(itertools.chain.from_iterable(wordnet_synset_examples))
    wordnet_synset_examples_tokenized = preprocess_data(wordnet_synset_examples, LANGUAGE)

    model = FastText(min_n=1, hs=True, size=100, window=5, alpha=0.001, min_alpha=0.0001, sg=True)
    model.build_vocab(sentences=wordnet_synset_examples_tokenized)
    model.train(sentences=wordnet_synset_examples_tokenized, total_examples=len(wordnet_synset_examples_tokenized), epochs=1)

    return model

def create_ml_dataset(df, nlp_fasttext):
    X = []
    Y = []

    for token, complexity in df[['token', 'complexity']].itertuples(index=False):
        if type(token) is not str:
            continue
        token = token.lower()
        token_embedding = nlp_fasttext.wv[token]

        X.append(token_embedding)
        Y.append(complexity)

    return X, Y

def create_ml_datasets(train_single_df, test_single_df, train_multi_df, test_multi_df, nlp_fasttext):
    return create_ml_dataset(train_single_df, nlp_fasttext), create_ml_dataset(test_single_df, nlp_fasttext), create_ml_dataset(train_multi_df, nlp_fasttext), create_ml_dataset(test_multi_df, nlp_fasttext)

In [3]:
train_single_df, test_single_df, train_multi_df, test_multi_df = load_dataframes()

nlp_fasttext = train_fasttext()

ml_single_train, ml_single_test, ml_multi_train, ml_multi_test = create_ml_datasets(train_single_df, test_single_df, train_multi_df, test_multi_df, nlp_fasttext)
ml_single_train_X, ml_single_train_Y = ml_single_train
ml_single_test_X, ml_single_test_Y = ml_single_test
ml_multi_train_X, ml_multi_train_Y = ml_multi_train
ml_multi_test_X, ml_multi_test_Y = ml_multi_test
ml_train_X = ml_single_train_X + ml_multi_train_X
ml_train_Y = ml_single_train_Y + ml_multi_train_Y

ml_train_X = np.array(ml_train_X)
ml_train_Y = np.array(ml_train_Y)

In [23]:
nlp_fasttext.save('models/FastText/model')

In [197]:
from sklearn.utils import all_estimators
from threading import Thread, Lock

import warnings
warnings.warn = lambda *args, **kwargs: None


regressor_results = {}
regressors_queued = 0

def launch_regressor(Regressor, regressor_name, X_train, Y_train, X_single_test, Y_single_test, X_multi_test, Y_multi_test):
    global regressors_queued
    global regressor_results
    
    try:
        mdl = Regressor()
        mdl.fit(X_train, Y_train)
        
        Y_single_test_pred = mdl.predict(X_single_test)
        Y_multi_test_pred = mdl.predict(X_multi_test)
        
        single_test_mse = np.mean(tf.keras.losses.MSE(Y_single_test, Y_single_test_pred))
        multi_test_mse = np.mean(tf.keras.losses.MSE(Y_multi_test, Y_multi_test_pred))
        mean_mse = (single_test_mse + multi_test_mse) / 2
        
        regressor_results[regressor_name] = {
            'Mean': mean_mse,
            'Single': single_test_mse,
            'Multi': multi_test_mse
        }
        
        print(regressor_name)
        print('Mean MSE:', mean_mse)
        print('Single-word expressions MSE:', single_test_mse)
        print('Multi-word expressions MSE:', multi_test_mse)
        print()
    except:
        pass

estimators = all_estimators(type_filter='regressor')

for regressor_name, Regressor in estimators:
    launch_regressor(Regressor, regressor_name, ml_train_X, ml_train_Y, ml_single_test_X, ml_single_test_Y, ml_multi_test_X, ml_multi_test_Y)

ARDRegression
Mean MSE: 0.022787291296267942
Single-word expressions MSE: 0.016758275903879075
Multi-word expressions MSE: 0.02881630668865681

AdaBoostRegressor
Mean MSE: 0.021642196915938102
Single-word expressions MSE: 0.019142057515973553
Multi-word expressions MSE: 0.02414233631590265

BaggingRegressor
Mean MSE: 0.02262677254552545
Single-word expressions MSE: 0.016062006191175284
Multi-word expressions MSE: 0.029191538899875618

BayesianRidge
Mean MSE: 0.022860627902460427
Single-word expressions MSE: 0.016759918286544318
Multi-word expressions MSE: 0.028961337518376536

CCA
Mean MSE: 0.02893077635741277
Single-word expressions MSE: 0.02801070597865537
Multi-word expressions MSE: 0.029850846736170174

DecisionTreeRegressor
Mean MSE: 0.036774562993142704
Single-word expressions MSE: 0.030920608751410047
Multi-word expressions MSE: 0.04262851723487536

DummyRegressor
Mean MSE: 0.02556034960890684
Single-word expressions MSE: 0.016815854084154667
Multi-word expressions MSE: 0.034304

In [198]:
optimal_regressor = min(list(regressor_results.items()), key=lambda x: x[1]['Mean']) 
print(optimal_regressor)

('HistGradientBoostingRegressor', {'Mean': 0.018519726659713985, 'Single': 0.014179329760857371, 'Multi': 0.022860123558570603})


In [18]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

hgbr = HistGradientBoostingRegressor(max_depth=10).fit(ml_train_X, ml_train_Y)

In [19]:
ml_single_test_Y_pred = hgbr.predict(np.array(ml_single_test_X))
ml_multi_test_Y_pred = hgbr.predict(np.array(ml_multi_test_X))

single_test_mse = np.mean(tf.keras.losses.MSE(ml_single_test_Y, ml_single_test_Y_pred))
multi_test_mse = np.mean(tf.keras.losses.MSE(ml_multi_test_Y, ml_multi_test_Y_pred))
mean_mse = (single_test_mse + multi_test_mse) / 2

print('Mean MSE:', mean_mse)
print('Single-word expressions MSE:', single_test_mse)
print('Multi-word expressions MSE:', multi_test_mse)

Mean MSE: 0.018673094400308544
Single-word expressions MSE: 0.014467996478526427
Multi-word expressions MSE: 0.022878192322090662


In [22]:
from joblib import dump

dump(hgbr, 'models/HistGradientBoostingRegressor/model')

['models/HistGradientBoostingRegressor/model']

In [201]:
import tensorflow as tf


def create_nn(X_train, Y_train):
    mdl = tf.keras.Sequential([
        tf.keras.layers.Dense(units=3, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid)
    ])
    
    mdl.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.01),
        loss=tf.keras.losses.MSE
    )

    mdl.fit(X_train, Y_train, epochs=100, batch_size=100, verbose=True)

    return mdl

In [202]:
nn = create_nn(ml_train_X, ml_train_Y)

Epoch 1/100
92/92 [==============================] - 0s 934us/step - loss: 0.0230
Epoch 2/100
92/92 [==============================] - 0s 915us/step - loss: 0.0204
Epoch 3/100
92/92 [==============================] - 0s 875us/step - loss: 0.0203
Epoch 4/100
92/92 [==============================] - 0s 875us/step - loss: 0.0202
Epoch 5/100
92/92 [==============================] - 0s 893us/step - loss: 0.0202
Epoch 6/100
92/92 [==============================] - 0s 894us/step - loss: 0.0201
Epoch 7/100
92/92 [==============================] - 0s 918us/step - loss: 0.0201
Epoch 8/100
92/92 [==============================] - 0s 918us/step - loss: 0.0201
Epoch 9/100
92/92 [==============================] - 0s 813us/step - loss: 0.0200
Epoch 10/100
92/92 [==============================] - 0s 831us/step - loss: 0.0200
Epoch 11/100
92/92 [==============================] - 0s 846us/step - loss: 0.0200
Epoch 12/100
92/92 [==============================] - 0s 870us/step - loss: 0.0200
Epoch 13/100


In [203]:
ml_single_test_Y_pred = nn.predict(np.array(ml_single_test_X))
ml_multi_test_Y_pred = nn.predict(np.array(ml_multi_test_X))

single_test_mse = np.mean(tf.keras.losses.MSE(ml_single_test_Y, ml_single_test_Y_pred))
multi_test_mse = np.mean(tf.keras.losses.MSE(ml_multi_test_Y, ml_multi_test_Y_pred))
mean_mse = (single_test_mse + multi_test_mse) / 2

print('Mean MSE:', mean_mse)
print('Single-word expressions MSE:', single_test_mse)
print('Multi-word expressions MSE:', multi_test_mse)

regressor_results['Neural Network'] = {
    'Mean': mean_mse,
    'Single': single_test_mse,
    'Multi': multi_test_mse
}

Mean MSE: 0.02377360686659813
Single-word expressions MSE: 0.018151967
Multi-word expressions MSE: 0.029395249


In [204]:
import copy
from tabulate import tabulate


sorted_regressor_results = list(regressor_results.items())

sorted_regressor_results.sort(key=lambda x: x[1]['Mean'])
sorted_regressor_results_mean = copy.deepcopy(sorted_regressor_results)

sorted_regressor_results.sort(key=lambda x: x[1]['Single'])
sorted_regressor_results_single = copy.deepcopy(sorted_regressor_results)

sorted_regressor_results.sort(key=lambda x: x[1]['Multi'])
sorted_regressor_results_multi = copy.deepcopy(sorted_regressor_results)

In [205]:
print(tabulate([(el[0], el[1]['Mean']) for el in sorted_regressor_results_mean], headers=['Regressor', 'Mean MSE']))

Regressor                        Mean MSE
-----------------------------  ----------
HistGradientBoostingRegressor   0.0185197
ExtraTreesRegressor             0.0185752
RandomForestRegressor           0.0191667
GradientBoostingRegressor       0.0198396
SVR                             0.0209658
NuSVR                           0.021139
AdaBoostRegressor               0.0216422
TheilSenRegressor               0.0226085
BaggingRegressor                0.0226268
LassoLarsIC                     0.0227862
ARDRegression                   0.0227873
LinearRegression                0.0228207
TransformedTargetRegressor      0.0228207
BayesianRidge                   0.0228606
OrthogonalMatchingPursuit       0.0234211
Neural Network                  0.0237736
HuberRegressor                  0.0237807
Lars                            0.0238535
LassoCV                         0.0239292
OrthogonalMatchingPursuitCV     0.0239704
ElasticNetCV                    0.0239765
RidgeCV                         0.0

In [206]:
print(tabulate([(el[0], el[1]['Single']) for el in sorted_regressor_results_single], headers=['Regressor', 'Single MSE']))

Regressor                        Single MSE
-----------------------------  ------------
HistGradientBoostingRegressor     0.0141793
RandomForestRegressor             0.0146046
GradientBoostingRegressor         0.0146074
ExtraTreesRegressor               0.0147745
SVR                               0.0154627
NuSVR                             0.0154717
BaggingRegressor                  0.016062
LinearSVR                         0.0165129
OrthogonalMatchingPursuit         0.0165924
MLPRegressor                      0.016627
LarsCV                            0.016672
SGDRegressor                      0.0167041
LassoLarsCV                       0.0167273
LassoLarsIC                       0.0167462
ARDRegression                     0.0167583
BayesianRidge                     0.0167599
Ridge                             0.016798
LassoCV                           0.0168139
DummyRegressor                    0.0168159
RadiusNeighborsRegressor          0.0168159
PoissonRegressor                  0.

In [207]:
print(tabulate([(el[0], el[1]['Multi']) for el in sorted_regressor_results_multi], headers=['Regressor', 'Multi MSE']))

Regressor                        Multi MSE
-----------------------------  -----------
ExtraTreesRegressor              0.0223758
HistGradientBoostingRegressor    0.0228601
RandomForestRegressor            0.0237288
AdaBoostRegressor                0.0241423
TheilSenRegressor                0.0249239
GradientBoostingRegressor        0.0250718
SVR                              0.0264689
NuSVR                            0.0268062
GaussianProcessRegressor         0.0268844
KNeighborsRegressor              0.0275589
LinearRegression                 0.0286467
TransformedTargetRegressor       0.0286467
ARDRegression                    0.0288163
LassoLarsIC                      0.0288262
BayesianRidge                    0.0289613
BaggingRegressor                 0.0291915
Neural Network                   0.0293952
CCA                              0.0298508
Lars                             0.0299896
OrthogonalMatchingPursuit        0.0302498
HuberRegressor                   0.0303989
OrthogonalM